In [ ]:
import pyarrow.parquet as pq
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

path = Path(r"D:\Project_end\New_world\my_project\logs\agent\RC_Tank\optimized__995.parquet")

# Load parquet
table = pq.read_table(path)
df = table.to_pandas()

df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

# Keys ที่ต้องการ plot
target_keys = [
    "loss_actor", "loss_critic",
    "q1_mean", "q2_mean",
    "entropy", "alpha",
    "tau", "action"
]

# 🔥 Fixed color mapping
color_map = {
    "loss_actor": "red",
    "loss_critic": "blue",
    "q1_mean": "green",
    "q2_mean": "purple",
    "entropy": "orange",
    "alpha": "brown",
    "tau": "pink",
    "action": "cyan"
}

rows, cols = 4, 2
wide, hight = 11.69, 8.27
fig, axes = plt.subplots(rows, cols, figsize=(wide/2, hight/2), sharex=False)

axes = axes.flatten()

for ax, key in zip(axes, target_keys):

    if key not in df["key"].unique():
        ax.text(0.5, 0.5, f"{key} (not found)", ha="center")
        ax.set_title(key)
        ax.grid(True)
        continue

    subset = df[df["key"] == key].reset_index(drop=True)

    x = subset.index
    y = subset["value"]

    # ใช้สีจาก color_map (ถ้า key ไม่มี map → ใช้ black)
    color = color_map.get(key, "black")

    ax.plot(x, y, color=color, label=key, linewidth=1.3)
    ax.set_title(key)
    ax.legend(loc="upper right", fontsize=8)
    ax.grid(True)

# Hide unused axes
for i in range(len(target_keys), len(axes)):
    axes[i].axis("off")

plt.tight_layout()
plt.show()


In [ ]:
import pyarrow.parquet as pq
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# โฟลเดอร์ที่เก็บไฟล์ parquet
folder = Path(r"D:\Project_end\New_world\my_project\logs\agent\RC_Tank")

# คีย์ที่ต้องการ plot
target_keys = [
    "loss_actor", "loss_critic",
    "q1_mean", "q2_mean",
    "entropy", "alpha",
    "tau", "action"
]

# 🔥 Fixed color mapping
color_map = {
    "loss_actor": "red",
    "loss_critic": "blue",
    "q1_mean": "green",
    "q2_mean": "purple",
    "entropy": "orange",
    "alpha": "brown",
    "tau": "pink",
    "action": "cyan"
}

# เตรียมเก็บค่าเฉลี่ยของทุกไฟล์
file_avgs = {key: [] for key in target_keys}

# 🔎 ไฟล์ทั้งหมดในโฟลเดอร์
files = sorted(folder.glob("*.parquet"))
print("Files found:", len(files))

# ---------------------------------------------------------
# อ่านทีละไฟล์ → คำนวณ avg → ลบทิ้งออกจาก RAM
# ---------------------------------------------------------
for fp in files:

    table = pq.read_table(fp)
    df = table.to_pandas()

    for key in target_keys:
        subset = df[df["key"] == key]["value"]
        if len(subset) > 0:
            file_avgs[key].append(subset.mean())
        else:
            file_avgs[key].append(None)

    del df
    del table

# ---------------------------------------------------------
# Plot Subplot 4×2 แบบเดิม แต่เป็น avg per file
# ---------------------------------------------------------
rows, cols = 4, 2
wide, hight = 11.69, 8.27
fig, axes = plt.subplots(rows, cols, figsize=(wide/1.7, hight/1.7), sharex=True)

axes = axes.flatten()

x = range(len(files))  # plot ต่อ file index

for ax, key in zip(axes, target_keys):
    
    y = file_avgs[key]

    color = color_map.get(key, "black")

    ax.plot(x, y, color=color, linewidth=1.3, label=key)
    ax.set_title(key)
    ax.legend(loc="upper right", fontsize=8)
    ax.grid(True)

# ซ่อน subplot ว่าง
for i in range(len(target_keys), len(axes)):
    axes[i].axis("off")

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import numpy as np
from matplotlib.gridspec import GridSpec

GROUPED_KEYS = [
    ["state", "setpoint"],   # row1 col1
    ["reward"],              # row1 col2
    ["action"],              # row2 full width
]

ANNOTATION_KEYS = ["episode", "done"]

COLOR_MAP = {
    "episode": "red",
    "setpoint": "blue",
    "state": "green",
    "reward": "purple",
    "done": "orange",
    "action": "cyan"
}

def plot_episode_mixed_layout(parquet_path):
    table = pq.read_table(parquet_path)
    df = table.to_pandas()

    if df.empty:
        print("Warning: DataFrame is empty.")
        return

    x = df.index

    # ─────────────────────────────────────────────
    # Create figure with GridSpec layout
    # Row1 → 2 columns
    # Row2 → 1 column that spans both
    # ─────────────────────────────────────────────
    fig = plt.figure(figsize=(10, 6))
    gs = GridSpec(2, 2, figure=fig, height_ratios=[1, 1.2])

    ax1 = fig.add_subplot(gs[0, 0])   # row1 col1
    ax2 = fig.add_subplot(gs[0, 1])   # row1 col2
    ax3 = fig.add_subplot(gs[1, :])   # row2 full width

    axes = [ax1, ax2, ax3]

    # ─────────────────────────────────────────────
    # Loop ตาม GROUPED_KEYS ของคุณ
    # ─────────────────────────────────────────────
    for ax, keys in zip(axes, GROUPED_KEYS):

        plotted = False
        
        for key in keys:
            if key not in df.columns:
                continue

            y = df[key]
            color = COLOR_MAP.get(key, "black")

            # numeric values
            if np.issubdtype(y.dtype, np.number):
                ax.plot(x, y, color=color, linewidth=1.6, label=key)
                plotted = True

            # list or vector
            elif isinstance(y.iloc[0], (list, np.ndarray)):
                y_arr = np.vstack(y.to_list())
                for d in range(y_arr.shape[1]):
                    ax.plot(
                        x, y_arr[:, d],
                        color=color,
                        alpha=0.7,
                        linewidth=1.4,
                        label=f"{key}[{d}]"
                    )
                plotted = True

        # Title annotation
        title_text = ""
        for annot_key in ANNOTATION_KEYS:
            if annot_key in df.columns:
                title_text += f"{annot_key}: {df[annot_key].iloc[-1]}  "

        ax.set_title(title_text.strip(), fontsize=10)

        if plotted:
            ax.legend(fontsize=8)
        ax.grid(True, linestyle="--", alpha=0.5)

    fig.tight_layout()
    plt.show()
plot_episode_mixed_layout(parquet_path=r"D:\Project_end\New_world\my_project\logs\episode\episode__990.parquet")

In [ ]:
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import numpy as np
from matplotlib.gridspec import GridSpec
from pathlib import Path
import pandas as pd


class EpisodePlotter:
    """
    A class that encapsulates episode plotting and statistical analysis
    for RL training logs stored in parquet format.
    """

    GROUPED_KEYS = [
        ["state", "setpoint"],   # row1 col1
        ["reward"],              # row1 col2
        ["action"],              # row2 full width
    ]

    COLOR_MAP = {
        "episode": "red",
        "setpoint": "blue",
        "state": "green",
        "reward": "purple",
        "done": "orange",
        "action": "red"
    }

    def __init__(self):
        pass

    # ----------------------------------------------------------
    # 1) Plot one episode
    # ----------------------------------------------------------
    def plot_episode(self, parquet_path):
        table = pq.read_table(parquet_path)
        df = table.to_pandas()

        if df.empty:
            print("Warning: DataFrame is empty.")
            return

        x = df.index
        ep_text = ""

        if "episode" in df.columns:
            ep_text += f"Episode: {df['episode'].iloc[-1]}   "
        if "done" in df.columns:
            ep_text += f"Done: {df['done'].iloc[-1]}"

        fig = plt.figure(figsize=(10, 6), constrained_layout=True)
        fig.suptitle(ep_text.strip(), fontsize=13, fontweight="bold")

        gs = GridSpec(2, 2, figure=fig, height_ratios=[1, 1.2])
        ax1 = fig.add_subplot(gs[0, 0])
        ax2 = fig.add_subplot(gs[0, 1])
        ax3 = fig.add_subplot(gs[1, :])
        axes = [ax1, ax2, ax3]
        titles = ["State vs Setpoint", "Reward", "Action"]

        for ax, keys, local_title in zip(axes, self.GROUPED_KEYS, titles):
            plotted = False

            for key in keys:
                if key not in df.columns:
                    continue

                y = df[key]
                color = self.COLOR_MAP.get(key, "black")

                if np.issubdtype(y.dtype, np.number):
                    ax.plot(df.index, y, color=color, linewidth=1.5, label=key)
                    plotted = True
                elif isinstance(y.iloc[0], (list, np.ndarray)):
                    y_arr = np.vstack(y.to_numpy())
                    for d in range(y_arr.shape[1]):
                        ax.plot(x, y_arr[:, d],
                                color=color, alpha=0.7,
                                linewidth=1.4, label=f"{key}[{d}]")
                    plotted = True

            ax.set_title(local_title, fontsize=11)
            ax.grid(True, linestyle="--", alpha=0.5)
            if plotted:
                ax.legend(fontsize=8, loc="upper right")

        plt.show()

    # ----------------------------------------------------------
    # 2) Load all actions
    # ----------------------------------------------------------
    def load_all_actions(self, folder_path):
        folder = Path(folder_path)
        files = sorted(folder.glob("*.parquet"))

        all_actions = []

        for f in files:
            table = pq.read_table(f)
            df = table.to_pandas()

            if "action" not in df.columns:
                continue

            if np.issubdtype(df["action"].dtype, np.number):
                all_actions.append(df["action"].to_numpy())
            elif isinstance(df["action"].iloc[0], (list, np.ndarray)):
                all_actions.append(np.vstack(df["action"].to_numpy()))

        if not all_actions:
            print("No 'action' column found.")
            return None

        return np.concatenate(all_actions, axis=0)

    # ----------------------------------------------------------
    # 3) Compute stats
    # ----------------------------------------------------------
    def compute_action_statistics(self, action_array):
        return {
            "mean": np.mean(action_array, axis=0),
            "std": np.std(action_array, axis=0),
            "min": np.min(action_array, axis=0),
            "max": np.max(action_array, axis=0)
        }

    # ----------------------------------------------------------
    # 4) Plot Action Distribution with Gradient Color
    # ----------------------------------------------------------
    def plot_action_distribution(self, action_array, bins=100, cmap_name="viridis"):
        """
        Plot histogram with per-bin gradient color.
        Supports both 1D and multi-dim actions.
        """
        action_array = np.array(action_array)

        # ------------------------------------------------------
        # Case A: 1D action
        # ------------------------------------------------------
        if action_array.ndim == 1:
            counts, bin_edges = np.histogram(action_array, bins=bins)
            widths = np.diff(bin_edges)

            cmap = plt.cm.get_cmap(cmap_name)
            gradient_vals = np.linspace(0, 1, bins)

            plt.figure(figsize=(8, 5))

            for i in range(bins):
                plt.bar(
                    bin_edges[i],
                    counts[i],
                    width=widths[i],
                    color=cmap(gradient_vals[i]),
                    edgecolor="black",
                    alpha=0.9
                )

            plt.title("Action Distribution (Gradient Histogram)")
            plt.xlabel("Action Value")
            plt.ylabel("Frequency")
            plt.grid(True, linestyle="--", alpha=0.5)
            plt.show()
            return

        # ------------------------------------------------------
        # Case B: Multi-dimensional actions
        # ------------------------------------------------------
        dim = action_array.shape[1]
        fig, axes = plt.subplots(dim, 1, figsize=(8, 3 * dim))
        axes = np.atleast_1d(axes)

        for d in range(dim):
            counts, bin_edges = np.histogram(action_array[:, d], bins=bins)
            widths = np.diff(bin_edges)

            cmap = plt.cm.get_cmap(cmap_name)
            gradient_vals = np.linspace(0, 1, bins)

            for i in range(bins):
                axes[d].bar(
                    bin_edges[i],
                    counts[i],
                    width=widths[i],
                    color=cmap(gradient_vals[i]),
                    edgecolor="black",
                    alpha=0.9
                )

            axes[d].set_title(f"Action[{d}] Distribution (Gradient)")
            axes[d].set_xlabel("Value")
            axes[d].set_ylabel("Freq")
            axes[d].grid(True, linestyle="--", alpha=0.5)

        plt.tight_layout()
        plt.show()


# -------------------------------
# Usage Example
# -------------------------------
plotter = EpisodePlotter()
actions = plotter.load_all_actions(r"D:\Project_end\New_world\my_project\logs\episode")
stats = plotter.compute_action_statistics(actions)
print(stats)
plotter.plot_action_distribution(actions)


In [ ]:
import pyarrow.parquet as pq
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt


class ParquetPlotter:
    """
    A unified plotter that can operate in two modes:
    1) single_file : plot raw values from a single parquet file
    2) avg_folder  : compute avg per file for multiple parquet files and plot
    """

    def __init__(
        self,
        target_keys,
        color_map=None,
        rows=4,
        cols=2,
        figsize=(6, 4),
    ):
        self.target_keys = target_keys
        self.color_map = color_map or {}
        self.rows = rows
        self.cols = cols
        self.figsize = figsize

    # ----------------------------------------------------------------------
    # Internal: plot helper
    # ----------------------------------------------------------------------
    def _create_subplots(self):
        fig, axes = plt.subplots(
            self.rows,
            self.cols,
            figsize=self.figsize,
            sharex=False
        )
        axes = axes.flatten()
        return fig, axes

    # ----------------------------------------------------------------------
    # Mode 1: plot จากไฟล์เดียว
    # ----------------------------------------------------------------------
    def plot_single_file(self, file_path: Path, save_path=None):

        table = pq.read_table(file_path)
        df = table.to_pandas()

        fig, axes = self._create_subplots()

        for ax, key in zip(axes, self.target_keys):

            if key not in df["key"].unique():
                ax.text(0.5, 0.5, f"{key} (not found)", ha="center")
                ax.set_title(key)
                ax.grid(True)
                continue

            subset = df[df["key"] == key].reset_index(drop=True)

            x = subset.index
            y = subset["value"]

            color = self.color_map.get(key, "black")

            ax.plot(x, y, color=color, linewidth=1.3, label=key)
            ax.set_title(key)
            ax.legend(loc="upper right", fontsize=8)
            ax.grid(True)

        # Hide unused axes
        for i in range(len(self.target_keys), len(axes)):
            axes[i].axis("off")

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=300)
        plt.show()

    # ----------------------------------------------------------------------
    # Mode 2: plot ค่าเฉลี่ยของทุกไฟล์ในโฟลเดอร์
    # ----------------------------------------------------------------------
    def plot_avg_folder(self, folder_path: Path, save_path=None):

        files = sorted(folder_path.glob("*.parquet"))
        print(f"Files found: {len(files)}")

        # เตรียมเก็บค่าเฉลี่ย
        file_avgs = {key: [] for key in self.target_keys}

        # อ่านไฟล์ทีละไฟล์
        for fp in files:
            table = pq.read_table(fp)
            df = table.to_pandas()

            for key in self.target_keys:
                subset = df[df["key"] == key]["value"]
                if len(subset) > 0:
                    file_avgs[key].append(subset.mean())
                else:
                    file_avgs[key].append(None)

            del df
            del table

        fig, axes = self._create_subplots()

        x = range(len(files))  # index ต่อไฟล์

        for ax, key in zip(axes, self.target_keys):
            y = file_avgs[key]
            color = self.color_map.get(key, "black")

            ax.plot(x, y, color=color, linewidth=1.3, label=key)
            ax.set_title(key)
            ax.legend(loc="upper right", fontsize=8)
            ax.grid(True)

        for i in range(len(self.target_keys), len(axes)):
            axes[i].axis("off")

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=300)
        plt.show()


In [ ]:
target_keys = [
    "loss_actor", "loss_critic",
    "q1_mean", "q2_mean",
    "entropy", "alpha",
    "tau", "action"
]

color_map = {
    "loss_actor": "red",
    "loss_critic": "blue",
    "q1_mean": "green",
    "q2_mean": "purple",
    "entropy": "orange",
    "alpha": "brown",
    "tau": "pink",
    "action": "cyan"
}

plotter = ParquetPlotter(
    target_keys,
    color_map=color_map,
    rows=4,
    cols=2,
    figsize=(11.69/2, 8.27/2)
)

plotter.plot_single_file(
    Path(r"D:\Project_end\New_world\my_project\logs\agent\RC_Tank\optimized__995.parquet"),
    save_path="single_plot.png"
)


In [ ]:
plotter = ParquetPlotter(
    target_keys,
    color_map=color_map,
    rows=4,
    cols=2,
    figsize=(11.69/1.7, 8.27/1.7)
)

plotter.plot_avg_folder(
    Path(r"D:\Project_end\New_world\my_project\logs\agent\RC_Tank"),
    save_path="avg_folder_plot.png"
)

In [ ]:
import pyarrow.parquet as pq
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt


class ParquetPlotter:
    """
    Auto-detecting plotter:
    - If given a parquet file → raw plot
    - If given a folder → average-per-file plot
    """

    def __init__(
        self,
        target_keys,
        color_map=None,
        rows=4,
        cols=2,
        figsize=(6, 4),
    ):
        self.target_keys = target_keys
        self.color_map = color_map or {}
        self.rows = rows
        self.cols = cols
        self.figsize = figsize

    # ----------------------------------------------------------------------
    # Internal: create subplot
    # ----------------------------------------------------------------------
    def _create_subplots(self):
        fig, axes = plt.subplots(
            self.rows,
            self.cols,
            figsize=self.figsize
        )
        return fig, axes.flatten()

    # ----------------------------------------------------------------------
    # Internal: detect mode
    # ----------------------------------------------------------------------
    def plot(self, path, save_path=None):
        """
        Auto-detect:
        - If path is a .parquet file → plot_single_file()
        - If path is a folder → plot_avg_folder()
        """
        path = Path(path)

        if path.is_file() and path.suffix == ".parquet":
            print(f"[MODE] Single file: {path.name}")
            return self.plot_single_file(path, save_path)

        elif path.is_dir():
            print(f"[MODE] Folder: {path}")
            return self.plot_avg_folder(path, save_path)

        else:
            raise ValueError("Path must be a .parquet file or a folder containing parquet files.")

    # ----------------------------------------------------------------------
    # Mode 1: plot from single parquet file
    # ----------------------------------------------------------------------
    def plot_single_file(self, file_path: Path, save_path=None):

        table = pq.read_table(file_path)
        df = table.to_pandas()

        fig, axes = self._create_subplots()

        for ax, key in zip(axes, self.target_keys):

            if key not in df["key"].unique():
                ax.text(0.5, 0.5, f"{key} (not found)", ha="center")
                ax.set_title(key)
                ax.grid(True)
                continue

            subset = df[df["key"] == key].reset_index(drop=True)

            x = subset.index
            y = subset["value"]
            color = self.color_map.get(key, "black")

            ax.plot(x, y, color=color, linewidth=1.3, label=key)
            ax.set_title(key)
            ax.legend(loc="upper right", fontsize=8)
            ax.grid(True)

        for i in range(len(self.target_keys), len(axes)):
            axes[i].axis("off")

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path, dpi=300)
        plt.show()

    # ----------------------------------------------------------------------
    # Mode 2: plot avg-per-file from folder
    # ----------------------------------------------------------------------
    def plot_avg_folder(self, folder_path: Path, save_path=None):

        files = sorted(folder_path.glob("*.parquet"))
        print(f"Files found: {len(files)}")

        file_avgs = {key: [] for key in self.target_keys}

        for fp in files:
            table = pq.read_table(fp)
            df = table.to_pandas()

            for key in self.target_keys:
                subset = df[df["key"] == key]["value"]
                file_avgs[key].append(subset.mean() if len(subset) else None)

            del df
            del table

        fig, axes = self._create_subplots()
        x = range(len(files))

        for ax, key in zip(axes, self.target_keys):
            y = file_avgs[key]
            color = self.color_map.get(key, "black")

            ax.plot(x, y, color=color, linewidth=1.3, label=key)
            ax.set_title(key)
            ax.legend(loc="upper right", fontsize=8)
            ax.grid(True)

        for i in range(len(self.target_keys), len(axes)):
            axes[i].axis("off")

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path, dpi=300)
        plt.show()


In [ ]:
target_keys = [
    "loss_actor", "loss_critic",
    "q1_mean", "q2_mean",
    "entropy", "alpha",
    "tau", "action"
]

color_map = {
    "loss_actor": "red",
    "loss_critic": "blue",
    "q1_mean": "green",
    "q2_mean": "purple",
    "entropy": "orange",
    "alpha": "brown",
    "tau": "pink",
    "action": "cyan"
}
plotter = ParquetPlotter(target_keys,
    color_map=color_map,
    rows=4,
    cols=2,
    figsize=(11.69/1.7, 8.27/1.7))
plotter.plot(r"D:\Project_end\New_world\my_project\logs\agent\RC_Tank")

In [ ]:
"""
Complete Parquet Plotting Utility
---------------------------------
Features:
- Auto-detect: plot single parquet file or average-per-file folder
- Load config from YAML
- Generate default YAML config
- Customizable subplot layout, colors, keys
"""

import yaml
import pyarrow.parquet as pq
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path


# ============================================================
# YAML CONFIG HANDLING
# ============================================================

def generate_default_config(path="config.yaml"):
    """
    Create a default YAML config with target keys and color map.
    """
    default_config = {
        "target_keys": [
            "loss_actor", "loss_critic",
            "q1_mean", "q2_mean",
            "entropy", "alpha",
            "tau", "action"
        ],

        "color_map": {
            "loss_actor": "#ff0000",
            "loss_critic": "#0000ff",
            "q1_mean": "#00aa00",
            "q2_mean": "#aa00aa",
            "entropy": "#ffa500",
            "alpha": "#8b4513",
            "tau": "#ff69b4",
            "action": "#00ffff",
        },

        "plot": {
            "rows": 4,
            "cols": 2,
            "figsize": [6, 4]
        }
    }

    path = Path(path)
    with open(path, "w", encoding="utf-8") as f:
        yaml.dump(default_config, f, allow_unicode=True, sort_keys=False)

    print(f"✔ YAML config created at: {path}")


def load_config(path):
    """
    Load YAML config file.
    """
    with open(path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)


# ============================================================
# PLOTTER CLASS
# ============================================================

class ParquetPlotter:
    """
    Auto-detecting plotter:
    - If path is a .parquet file → plot_single_file()
    - If path is a folder → plot_avg_folder()
    """

    def __init__(
        self,
        target_keys,
        color_map=None,
        rows=4,
        cols=2,
        figsize=(6, 4)
    ):
        self.target_keys = target_keys
        self.color_map = color_map or {}
        self.rows = rows
        self.cols = cols
        self.figsize = figsize

    # --------------------------------------------------------
    # Static constructor from YAML
    # --------------------------------------------------------
    @staticmethod
    def from_yaml(path):
        cfg = load_config(path)
        return ParquetPlotter(
            target_keys=cfg["target_keys"],
            color_map=cfg.get("color_map", {}),
            rows=cfg["plot"]["rows"],
            cols=cfg["plot"]["cols"],
            figsize=tuple(cfg["plot"]["figsize"])
        )

    # --------------------------------------------------------
    # Auto-detect mode
    # --------------------------------------------------------
    def plot(self, path, save_path=None):
        """
        Auto-detect:
        - .parquet file → single file mode
        - folder → average-per-file mode
        """
        path = Path(path)

        if path.is_file() and path.suffix == ".parquet":
            print(f"[MODE] Single File: {path.name}")
            return self.plot_single_file(path, save_path)

        elif path.is_dir():
            print(f"[MODE] Folder: {path}")
            return self.plot_avg_folder(path, save_path)

        else:
            raise ValueError("Path must be a parquet file or folder containing parquet files.")

    # --------------------------------------------------------
    # Plot: Single parquet file
    # --------------------------------------------------------
    def plot_single_file(self, file_path, save_path=None):
        table = pq.read_table(file_path)
        df = table.to_pandas()

        fig, axes = self._create_subplot_grid()

        for ax, key in zip(axes, self.target_keys):

            if key not in df["key"].unique():
                ax.text(0.5, 0.5, f"{key} (not found)", ha="center")
                ax.set_title(key)
                ax.grid(True)
                continue

            subset = df[df["key"] == key].reset_index(drop=True)

            x = subset.index
            y = subset["value"]
            color = self.color_map.get(key, "black")

            ax.plot(x, y, color=color, linewidth=1.3, label=key)
            ax.set_title(key)
            ax.legend(loc="upper right", fontsize=8)
            ax.grid(True)

        self._hide_unused_axes(axes)
        self._finalize_plot(save_path)

    # --------------------------------------------------------
    # Plot: average per file in folder
    # --------------------------------------------------------
    def plot_avg_folder(self, folder_path, save_path=None):

        files = sorted(folder_path.glob("*.parquet"))
        print(f"Files found: {len(files)}")

        file_avgs = {key: [] for key in self.target_keys}

        for fp in files:
            table = pq.read_table(fp)
            df = table.to_pandas()

            for key in self.target_keys:
                subset = df[df["key"] == key]["value"]
                file_avgs[key].append(subset.mean() if len(subset) else None)

        fig, axes = self._create_subplot_grid()
        x = range(len(files))

        for ax, key in zip(axes, self.target_keys):
            y = file_avgs[key]
            color = self.color_map.get(key, "black")

            ax.plot(x, y, color=color, linewidth=1.3, label=key)
            ax.set_title(key)
            ax.legend(loc="upper right", fontsize=8)
            ax.grid(True)

        self._hide_unused_axes(axes)
        self._finalize_plot(save_path)

    # ============================================================
    # INTERNAL UTILITIES
    # ============================================================

    def _create_subplot_grid(self):
        fig, axes = plt.subplots(self.rows, self.cols, figsize=self.figsize)
        return fig, axes.flatten()

    def _hide_unused_axes(self, axes):
        for i in range(len(self.target_keys), len(axes)):
            axes[i].axis("off")

    def _finalize_plot(self, save_path):
        plt.tight_layout()
        if save_path:
            plt.savefig(save_path, dpi=300)
        plt.show()


# ============================================================
# USAGE HELP (Optional)
# ============================================================
if __name__ == "__main__":
    print("Example usage:")
    print("  generate_default_config('config.yaml')")
    print("  plotter = ParquetPlotter.from_yaml('config.yaml')")
    print("  plotter.plot('path_to_file_or_folder')")


In [ ]:
generate_default_config("plot.yaml")

In [ ]:
plotter = ParquetPlotter.from_yaml("plot.yaml")
plotter.plot(r"D:\Project_end\New_world\my_project\logs\agent\RC_Tank")

In [ ]:
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import numpy as np
from pathlib import Path

class EpisodePlotter:
    """
    EpisodePlotter with multi-dimensional state support.
    Plots:
    - final state per dimension
    - final setpoint distribution
    - max reward distribution
    - action distribution
    """

    COLOR_MAP = {
        "episode": "red",
        "setpoint": "blue",
        "state": "green",
        "reward": "purple",
        "done": "orange",
        "action": "red"
    }

    def __init__(self):
        pass

    # ----------------------------------------
    # Extract summary statistics from folder
    # ----------------------------------------
    def extract_folder_statistics(self, folder_path):
        folder = Path(folder_path)
        files = sorted(folder.glob("*.parquet"))

        final_states = []
        final_setpoints = []
        max_rewards = []
        all_actions = []

        for f in files:
            df = pq.read_table(f).to_pandas()
            if df.empty:
                continue

            # State
            if "state" in df.columns:
                val = df["state"].iloc[-1]
                final_states.append(np.array(val) if isinstance(val, (list, np.ndarray)) else np.array([val]))

            # Setpoint
            if "setpoint" in df.columns:
                val = df["setpoint"].iloc[-1]
                final_setpoints.append(np.array(val) if isinstance(val, (list, np.ndarray)) else np.array([val]))

            # Reward
            if "reward" in df.columns:
                max_rewards.append(df["reward"].max())

            # Action
            if "action" in df.columns:
                col = df["action"]
                if np.issubdtype(col.dtype, np.number):
                    all_actions.append(col.to_numpy())
                else:
                    all_actions.append(np.vstack(col.to_numpy()))

        all_actions = np.concatenate(all_actions, axis=0) if all_actions else None

        return {
            "final_state": np.array(final_states, dtype=object),
            "final_setpoint": np.array(final_setpoints, dtype=object),
            "max_reward": np.array(max_rewards),
            "all_actions": all_actions
        }

    # ----------------------------------------
    # Plot distribution overview (state separated by dimension)
    # ----------------------------------------
    def plot_distribution_overview(self, stats, rows=2, cols=2, figsize=(10, 6)):
        """
        stats = output of extract_folder_statistics()
        """
        fig, axes = plt.subplots(rows, cols, figsize=figsize)
        axes = axes.flatten()

        # ---------------- State (per dimension) ----------------
        if len(stats["final_state"]) > 0:
            # Determine max dimensions
            max_dim = max([v.shape[0] for v in stats["final_state"]])
            for dim in range(max_dim):
                ax = axes[dim]
                # Collect dim-th value from each state
                values = [v[dim] if dim < v.shape[0] else np.nan for v in stats["final_state"]]
                ax.hist(values, bins=30, alpha=0.8, color=self.COLOR_MAP["state"])
                ax.set_title(f"Final State Dim {dim}")
                ax.grid(True, linestyle="--", alpha=0.5)

        # ---------------- Setpoint ----------------
        ax_idx = max_dim
        if ax_idx < len(axes) and len(stats["final_setpoint"]) > 0:
            sp_values = [np.mean(v) if len(v) > 1 else v[0] for v in stats["final_setpoint"]]
            axes[ax_idx].hist(sp_values, bins=30, alpha=0.8, color=self.COLOR_MAP["setpoint"])
            axes[ax_idx].set_title("Final Setpoint")
            axes[ax_idx].grid(True, linestyle="--", alpha=0.5)
            ax_idx += 1

        # ---------------- Reward ----------------
        if ax_idx < len(axes) and len(stats["max_reward"]) > 0:
            axes[ax_idx].hist(stats["max_reward"], bins=30, alpha=0.8, color=self.COLOR_MAP["reward"])
            axes[ax_idx].set_title("Max Reward")
            axes[ax_idx].grid(True, linestyle="--", alpha=0.5)
            ax_idx += 1

        # ---------------- Action ----------------
        if ax_idx < len(axes) and stats["all_actions"] is not None:
            arr = stats["all_actions"]
            if arr.ndim == 1:
                axes[ax_idx].hist(arr, bins=50, alpha=0.8, color=self.COLOR_MAP["action"])
            else:
                axes[ax_idx].hist(arr.mean(axis=1), bins=50, alpha=0.8, color=self.COLOR_MAP["action"])
            axes[ax_idx].set_title("Action Distribution")
            axes[ax_idx].grid(True, linestyle="--", alpha=0.5)

        # Hide unused axes
        for i in range(ax_idx + 1, len(axes)):
            axes[i].axis("off")

        plt.tight_layout()
        plt.show()


In [ ]:
plotter = EpisodePlotter()
stats = plotter.extract_folder_statistics(r"D:\Project_end\New_world\my_project\logs\episode")

# สามารถปรับ rows/cols ได้ตามจำนวน state dimension + 3
plotter.plot_distribution_overview(stats, rows=3, cols=2, figsize=(12, 8))
